## GPUs: From Pipelines to Accelerators

A GPU is:
* Specialized hardare for graphics rendering
* Optimized for floating point transformations on 
    * 3-d models (representations of physical objects)
    * 2-d dense arrays (presentation) 
    
    
A little history:
  * 